In [1]:
import glob
import os


# Reminder to install s3fs to read files from aws
import s3fs

# import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import xarray as xr

import seaborn as sns

# import scanpy.api as sc

# import holoviews as hv
# hv.extension('bokeh')
# hv.archive.auto()

%matplotlib inline

In [2]:
! aws s3 ls s3://olgabot-maca/facs/sourmash_compare/ > aws_s3_ls_tabula_muris_compare.txt

aws_s3_ls = pd.read_table('aws_s3_ls_tabula_muris_compare.txt', 
                          delim_whitespace=True, header=None, 
                          names=['date', 'time', 'bytes', 'basename'])
print(aws_s3_ls.shape)
aws_s3_ls.head()

(1040, 4)


,date,time,bytes,basename
0,2018-09-14,22:08:28,19045449,trim=false_scaled=1000_dna_ksize=12.csv
1,2018-08-10,11:01:24,19395540,trim=false_scaled=1000_dna_ksize=15.csv
2,2018-09-14,22:01:01,20133019,trim=false_scaled=1000_dna_ksize=18.csv
3,2018-09-12,23:40:02,10499576,trim=false_scaled=1000_dna_ksize=21.csv
4,2018-09-15,05:15:04,20258815,trim=false_scaled=1000_dna_ksize=24.csv


In [3]:
sorted(aws_s3_ls.date.unique())

['2018-08-09',
 '2018-08-10',
 '2018-08-11',
 '2018-08-12',
 '2018-09-12',
 '2018-09-13',
 '2018-09-14',
 '2018-09-15',
 '2018-09-16',
 '2018-09-18',
 '2018-09-19',
 '2018-09-20']

In [4]:
prefix = 's3://olgabot-maca/facs/sourmash_compare/'

In [5]:
df = pd.read_csv(prefix + aws_s3_ls.query('date == "2018-08-12"').iloc[0]['basename'])
df.iloc[:5, :5]

,N9-B002428-3_38_F-1-1|tissue:Heart|subtissue:LA|cell_ontology_class:fibroblast|free_annotation:NA,E5-D042479-3_8_M-1-1|tissue:Marrow|subtissue:B-cells|cell_ontology_class:precursor_B_cell|free_annotation:pre-B_cell_(Philadelphia_nomenclature),L1-D042475-3_9_M-1-1|tissue:Trachea|subtissue:NA|cell_ontology_class:mesenchymal_cell|free_annotation:NA,P12-MAA000596-3_9_M-1-1|tissue:Marrow|subtissue:KLS|cell_ontology_class:hematopoietic_precursor_cell|free_annotation:NA,I10-MAA000385-3_10_M-1-1|tissue:Fat|subtissue:SCAT|cell_ontology_class:myeloid_cell|free_annotation:NA
0,1.000000,0.455388,0.738791,0.609054,0.497532
1,0.455388,1.000000,0.542118,0.556670,0.519706
2,0.738791,0.542118,0.999999,0.731973,0.588434
3,0.609054,0.556670,0.731973,1.000000,0.647301
4,0.497532,0.519706,0.588434,0.647301,1.000000


In [6]:
df = pd.read_csv(prefix + aws_s3_ls.query('date == "2018-09-12"').iloc[0]['basename'])
df.iloc[:5, :5]

,G12-MAA000496-3_10_M-1-1|tissue:Trachea|subtissue:NA|cell_ontology_class:mesenchymal_cell|free_annotation:NA,F12-D045140-3_38_F-1-1|tissue:Marrow|subtissue:Granulocytes|cell_ontology_class:granulocyte|free_annotation:NA,N9-MAA000617-3_10_M-1-1|tissue:Brain_Myeloid|subtissue:Striatum|cell_ontology_class:microglial_cell|free_annotation:NA,F20-MAA000934-3_9_M-1-1|tissue:Fat|subtissue:SCAT|cell_ontology_class:mesenchymal_stem_cell_of_adipose|free_annotation:mesenchymal_progenitor,A8-MAA000609-3_8_M-1-1|tissue:Thymus|subtissue:NA|cell_ontology_class:immature_T_cell|free_annotation:DN4-DP_in_transition_Cd69_negative_thymocytes
0,1.000000,0.027854,0.044296,0.081313,0.040397
1,0.027854,1.000000,0.028868,0.030979,0.023340
2,0.044296,0.028868,1.000000,0.054794,0.036477
3,0.081313,0.030979,0.054794,1.000000,0.035593
4,0.040397,0.023340,0.036477,0.035593,1.000000


In [7]:
df = pd.read_csv(prefix + aws_s3_ls.query('date == "2018-09-15"').iloc[0]['basename'])
df.iloc[:5, :5]

,M20-B002775-3_39_F-1-1|tissue:Kidney|subtissue:NA|cell_ontology_class:epithelial_cell_of_proximal_tubule|free_annotation:NA,N9-B002011-3_38_F-1-1|tissue:Heart|subtissue:RA|cell_ontology_class:fibroblast|free_annotation:NA,L17-B002423-3_39_F-1-1|tissue:Heart|subtissue:RV|cell_ontology_class:fibroblast|free_annotation:NA,N16-MAA001539-3_56_F-1-1|tissue:Large_Intestine|subtissue:Distal|cell_ontology_class:enterocyte_of_epithelium_of_large_intestine|free_annotation:Enterocyte_(Distal),B21-D042474-3_8_M-1-1|tissue:Trachea|subtissue:NA|cell_ontology_class:endothelial_cell|free_annotation:NA
0,1.000000,0.030867,0.041741,0.026250,0.029224
1,0.030867,1.000000,0.059843,0.020217,0.043646
2,0.041741,0.059843,1.000000,0.027921,0.046250
3,0.026250,0.020217,0.027921,1.000000,0.021005
4,0.029224,0.043646,0.046250,0.021005,1.000000


In [8]:
# df = pd.read_csv(prefix + aws_s3_ls.query('date == "2018-09-16"').iloc[0]['basename'])
# df.iloc[:5, :5]

CSVs created from 2018-09-12 to 2018-09-15 are all with the `--ignore-abundance` flag

In [9]:
from tqdm import tqdm

In [ ]:

def extract_cell_metadata(columns):
    cell_metadata = pd.Series(columns).str.split('|', expand=True)
#     cell_metadata.index = columns
    cell_metadata.columns = ['cell_id', 'tissue', 
                             'subtissue', 'cell_ontology_class', 
                             'free_annotation']
    cell_metadata = cell_metadata.set_index('cell_id')
    cell_metadata.index.name = 'cell_id'
    cell_metadata = cell_metadata.applymap(lambda x: x.split(':')[-1])
#     print('\tcell_metadata.shape', cell_metadata.shape)
#     cell_metadata = cell_metadata.sort_index()
#     cell_metadata.head()
    return cell_metadata



dfs = {}

metadatas = []

def to_key_value_pair(attribute):
    if len(attribute) > 1:
        try:
            return attribute[0], int(attribute[1])
        except ValueError:
            return attribute[0], attribute[1] 
    else:
        return 'comparison_sequence', attribute[0]



def extract_experiment_metadata(basename):
    key = basename.split('.csv')[0]
    split = key.split('_')
    attributes = [x.split('=') for x in split]
    attributes = dict(to_key_value_pair(x) for x in attributes)
    return key, attributes


ignore_abundance_dates = ['2018-09-12',
 '2018-09-13',
 '2018-09-14',
 '2018-09-15']

experiment_metadatas = {}

cell_metadata = pd.DataFrame()

max_cells = 1000
tidy_dfs = []

for i, row in tqdm(aws_s3_ls.iterrows()):
    basename = row['basename']
    csv = f'{prefix}{basename}'
    df = pd.read_csv(csv)
#     print(f'{basename}\tdf.shape: {df.shape}')
    
    key, attributes = extract_experiment_metadata(basename)
    experiment_metadatas[key] = attributes

    metadata = extract_cell_metadata(df.columns)
    
    mask = np.triu(np.ones(df.shape)).astype(np.bool)

    df.index = metadata.index.copy()
    df.columns = metadata.index.copy()
    
    df = df.where(mask)
    df.index.name = 'cell_id'
    df.columns.name = 'other_cell'
    tidy = df.stack().reset_index()
    tidy['experiment'] = key
    
    ignore_abundance = row['date'] in ignore_abundance_dates
    tidy['ignore_abundance'] = ignore_abundance
#     tidy = tidy.rename(columns={'level_0': 'cell_id', 'level_1': 'other_cell'})
        
    tidy_dfs.append(tidy)

    # Add new cells to metadata
    if len(df.index.difference(cell_metadata.index)) > 0:
        cell_metadata = cell_metadata.append(metadata)
        cell_metadata = cell_metadata.loc[~cell_metadata.index.duplicated()]
        print(f'\tcell_metadata.shape, {cell_metadata.shape}')



1it [00:01,  1.06s/it]

	cell_metadata.shape, (997, 4)


21it [00:17,  1.17it/s]

	cell_metadata.shape, (1000, 4)


441it [10:14,  1.39s/it]

In [ ]:
# cell_metadata = pd.concat(metadatas)
# print(cell_metadata.shape)
# cell_metadata = cell_metadata.loc[~cell_metadata.index.duplicated()]
cell_metadata = cell_metadata.sort_index()
print(cell_metadata.shape)
cell_metadata.head()

In [ ]:
experiment_metadata = pd.DataFrame(experiment_metadatas).T
experiment_metadata['trim'] = experiment_metadata['trim'].map(lambda x: x == 'true')
experiment_metadata['ksize'] = experiment_metadata['ksize'].astype(int)
experiment_metadata['scaled'] = experiment_metadata['scaled'].astype(int)
print(experiment_metadata.shape)
experiment_metadata.head()

In [ ]:
# csv_prefix = '/mnt/data/tabula-muris-sourmash-compare'

In [ ]:
import s3fs

def write_s3(df, filename, fmt='csv', **kwargs):
    fs = s3fs.S3FileSystem(anon=False)
    if fmt == 'csv':
        # csv is a text format
        with fs.open(filename, 'w') as f:
            return df.to_csv(f, **kwargs)
    elif fmt == 'parquet':
        # Parquet is a binary format and needs the "b" flag
        with fs.open(filename, 'wb') as f:
            return df.to_parquet(f, engine='pyarrow', **kwargs)
        

In [ ]:
csv_prefix = 's3://olgabot-maca/facs/sourmash_compare_combined/'

In [ ]:
    # %load_ext autoreload
    # %autoreload 2

In [ ]:
pd.__file__

In [ ]:
write_s3(experiment_metadata, f'{csv_prefix}experiment-metadata.csv', 'csv')

write_s3(cell_metadata, f'{csv_prefix}cell-metadata.csv', 'csv')


In [ ]:
similarity_scaled_subset = similarity.query()

In [ ]:
%time similarity = pd.concat(tidy_dfs, ignore_index=True)
print(similarity.shape)
similarity.head()

In [ ]:
%time similarity = similarity.rename(columns={0:'similarity'})

In [ ]:
%time write_s3(similarity, f'{csv_prefix}similarity.csv', 'csv')

In [ ]:
%time write_s3(similarity, f'{csv_prefix}similarity.parquet', 'parquet')

In [ ]:
! aws s3 ls --human-readable $csv_prefix

In [ ]:
# experiment_metadata.to_csv(f'{csv_prefix}-experiment-metadata.csv')

In [ ]:
cell_metadata.to_csv(f'{csv_prefix}-cell-metadata.csv')

In [ ]:
%time compare_results.to_parquet(f'{csv_prefix}-similarity.parquet')

## Save subset with only a few scalings

In [ ]:
sorted(experiment_metadata.scaled.unique())

In [ ]:
scaleds = [100, 500, 1000, 5000]

experiment_metadata_subset = experiment_metadata.loc[experiment_metadata.scaled.isin(scaleds)]
experiment_metadata_subset.head()

In [ ]:
%time similarity_subset = similarity.query('experiment in @experiment_metadata_subset.index')
print(similarity_subset.shape)
similarity_subset.head()

In [ ]:
%time write_s3(experiment_metadata_subset, f'{csv_prefix}experiment-metadata-scaled-subset.csv', 'csv')


In [ ]:
similarity_subset.dtypes

In [ ]:
%time write_s3(similarity_subset, f'{csv_prefix}similarity-scaled-subset.parquet', 'parquet')

In [ ]:
# attrs = experiment_metadata.T.to_dict()
# attrs

In [ ]:
ds = 1-xr.Dataset(dfs)
ds

In [ ]:
ds.to_netcdf('/mnt/data/tabula-muris-sourmash-compare.netcdf')

In [ ]:
ls -lh /mnt/data/

## Read full, normalized dataset to see where the cells lie

In [ ]:
tabula_muris = sc.read('/mnt/data/tabula-muris-facs.h5ad')
tabula_muris

In [ ]:
rows = tabula_muris.obs.index.isin(cell_metadata.index)

facs_1000cell_cheating = tabula_muris[rows, :]


sc.pl.umap(facs_1000cell_cheating, 'cell_ontology_class')

## Re-load original, unnormalized count matrices

In [ ]:
import scanpy

results_file = '/mnt/data/tabula-muris-facs_1000cell_subset.h5ad'


tm_facs_metadata = pd.read_csv('/mnt/data/czbiohub-tabula-muris/TM_facs_metadata.csv')
tm_facs_metadata = tm_facs_metadata.set_index('cell')

# Replace dots with dashes
index = tm_facs_metadata.index.str.replace('.', '-')
tm_facs_metadata.index = index

tm_facs_data = scanpy.anndata.read_h5ad('/mnt/data/czbiohub-tabula-muris/TM_facs_mat.h5ad')

tm_facs_data.obs.index = index
tm_facs_data.obs = tm_facs_metadata
tm_facs_data

In [ ]:
rows = tm_facs_data.obs.index.isin(cell_metadata.index)

facs_1000cell = tm_facs_data[rows, :]
sc.tl.pca(facs_1000cell)

sc.pl.pca_variance_ratio(facs_1000cell, log=True)

In [ ]:
sc.pp.neighbors(facs_1000cell, n_neighbors=5, n_pcs=19)
sc.tl.umap(facs_1000cell)

sc.pl.umap(facs_1000cell, 'cell_ontology_class')

In [ ]:
len(facs_1000cell.obs.cell_ontology_class.unique())

In [ ]:
facs_1000cell.write(results_file)

In [ ]:
def embedding_to_df(embedding, metadata):
    embedding_df = pd.DataFrame(embedding, index=metadata.index, columns=['UMAP_1', 'UMAP_2'])

    sample_id_to_cell_ontology_class = pd.DataFrame(metadata[['tissue', 'cell_ontology_class']].values, 
                                                 index=metadata.index, 
                                                    columns=['tissue', 'cell_ontology_class'])
    sample_id_to_cell_ontology_class['mouse_sex'] = sample_id_to_cell_ontology_class.index.map(lambda x: x.split("_")[-1][0])
    sample_id_to_cell_ontology_class.head()

    embedding_metadata = embedding_df.join(sample_id_to_cell_ontology_class)
    embedding_metadata.index.name = 'cell_id'
    return embedding_metadata

def embedding_to_hv_table(embedding_metadata):

    key_dimensions = [('cell_ontology_class', 'Cell ontology class'),
                      ('tissue', "Organ"), ('mouse_sex', 'Mouse Sex')]
    value_dimensions = [("UMAP_1", 'UMAP 1'), ("UMAP_2", "UMAP 2")]
    embedding_hv = hv.Table(embedding_metadata.reset_index(), key_dimensions, value_dimensions)
    return embedding_hv

In [ ]:
alignment_umap_df = embedding_to_df(facs_1000cell.obsm['X_umap'], facs_1000cell.obs)
alignment_umap_hv = embedding_to_hv_table(alignment_umap_df)
alignment_umap_hv

### Define a reasonable looking color cycle

In [ ]:
cat20c = hv.Cycle("Category20c")
cat20b = hv.Cycle("Category20b")

new_palette = cat20b.values + cat20c.values

color_cycle = hv.Cycle(new_palette)

### Plot interactive version of umap

In [ ]:
%%opts Scatter [width=400 height=400 tools=['hover']]
%%opts Scatter (color=color_cycle)

scatters = alignment_umap_hv.to.scatter('UMAP 1', 'UMAP 2', groupby='Cell ontology class')
scatters.overlay('Cell ontology class')


## Get median ranks of cells adjacent in gene count space

In [ ]:
cell_ids = facs_1000cell.obs.index
facs_1000cell_adjacency_matrix = pd.DataFrame(facs_1000cell.uns['neighbors']['connectivities'].todense(),
                                          index=cell_ids, 
                                          columns=cell_ids)
# adjacency_matrix = 
facs_1000cell_adjacency_matrix.head()

In [ ]:
(facs_1000cell_adjacency_matrix > 0).sum().head()

In [ ]:
def median_neighbor_rank(jaccard_distances, adjacency_matrix, dim='cell_id', 
                         ascending=True):
    
    index = jaccard_distances.coords.indexes.get(dim)
    da_as_df = pd.DataFrame(jaccard_distances.values, index=index, columns=index)
    jaccard_ranks = da_as_df.rank(ascending=ascending)

    median_ranks = jaccard_ranks[adjacency_matrix > 0].median()
    return median_ranks

median_ranks = median_neighbor_rank(ds['trim=false_scaled=1000_dna_ksize=12'],
                                    facs_1000cell_adjacency_matrix)
# sns.distplot(median_ranks)
median_ranks.head()

In [ ]:
sns.distplot(median_ranks.dropna())

In [ ]:
median_ranks_all = ds.apply(median_neighbor_rank, adjacency_matrix=facs_1000cell_adjacency_matrix)
median_ranks_all

In [ ]:
median_ranks_df = median_ranks_all.to_dataframe().T.unstack().reset_index()
median_ranks_df = median_ranks_df.rename(
    columns={"level_1": 'experiment_id', 0: 'nn_median_rank'})
median_ranks_df.head()

In [ ]:
median_ranks_metadata = median_ranks_df.join(experiment_metadata, on='experiment_id') 
print(median_ranks_metadata.shape)
median_ranks_metadata.head()

In [ ]:
ksize_order = sorted(median_ranks_metadata['ksize'].unique())
ksize_order

In [ ]:
scaled_order = sorted(median_ranks_metadata['scaled'].unique())
scaled_order

In [ ]:
median_ranks_metadata.dtypes

In [ ]:
g = sns.FacetGrid(median_ranks_metadata, hue='scaled', col='trim', 
                  size=6, hue_order=scaled_order, palette='viridis')
g = g.map(sns.pointplot, 'ksize', 'nn_median_rank', lowess=True, order=ksize_order)
g = g.add_legend()

In [ ]:
g = sns.FacetGrid(median_ranks_metadata, hue='scaled', col='trim', 
                  height=6, hue_order=scaled_order, palette='viridis')
g.map(sns.regplot, 'ksize', 'nn_median_rank', lowess=True)
g.add_legend()
g.set(xlim=(3, 60))

In [ ]:
sns.regplot?

In [ ]:
sns.pointplot?

In [ ]:
scales = [100, 500, 1000, 2000, 5000]

In [ ]:
median_ranks_metadata_subset_scales = median_ranks_metadata.query('scaled in @scales')
median_ranks_metadata_subset_scales = median_ranks_metadata_subset_scales.dropna()
print(median_ranks_metadata_subset_scales.shape)
median_ranks_metadata_subset_scales.head()

In [ ]:
g = sns.FacetGrid(median_ranks_metadata_subset_scales, hue='scaled', col='trim', 
                  height=6, palette='Set1')
g = g.map(sns.pointplot, 'ksize', 'nn_median_rank', lowess=True, order=ksize_order)
g = g.add_legend()

In [ ]:
g = sns.FacetGrid(median_ranks_metadata_subset_scales, hue='scaled', col='trim', 
                  height=6, palette='Set1')
g.map(sns.regplot, 'ksize', 'nn_median_rank', lowess=True)
g.add_legend()

# For some reason scatter=False doesn't work so you have to manually remove them
ax = g.axes[0][0]
ax.collections = []

In [ ]:
sns.regplot?

In [ ]:
with sns.axes_style('whitegrid'):
    g = sns.FacetGrid(median_ranks_metadata_subset_scales, hue='scaled', col='trim', 
                      height=6, palette='Set1')
    g.map(sns.regplot, 'ksize', 'nn_median_rank', lowess=True, scatter=False, 
          line_kws=dict(alpha=0.75))
    g.add_legend()
    g.set(xlim=(0, 60))

In [ ]:
with sns.axes_style('whitegrid'):
    g = sns.FacetGrid(median_ranks_metadata_subset_scales, hue='scaled', col='trim', 
                      height=6, palette='Set1')
    g.map(sns.regplot, 'ksize', 'nn_median_rank', lowess=True, scatter=False, 
          line_kws=dict(alpha=0.75))
    g.add_legend()
    g.set(xlim=(20, 34), xticks=range(21, 35, 3))


In [ ]:
fig, ax = plt.subplots(figsize=(20, 4))

sns.violinplot('ksize', 'nn_median_rank', data=median_ranks_metadata_subset_scales, 
               hue='scaled', palette='viridis', scale='width')
sns.despine()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 4))

sns.boxplot('ksize', 'nn_median_rank', data=median_ranks_metadata_subset_scales, 
               hue='scaled', palette='viridis')
sns.despine()

In [ ]:
# g = sns.FacetGrid(median_ranks_metadata_subset, hue='scaled', col='trim', 
# #                   row='scaled',
#                   height=6, palette='viridis', aspect=2)
# g.map(sns.boxplot, 'ksize', 'nn_median_rank')
# g.add_legend()

In [ ]:
# g = sns.FacetGrid(median_ranks_metadata_subset, hue='scaled', 
#                   row='ksize', sharex=True, sharey=True,
#                   height=1, palette='viridis', aspect=2)
# g.map(sns.distplot, 'nn_median_rank', kde=False)
# g.add_legend()

In [ ]:
ksizes = list(range(21, 34, 3))
ksizes

In [ ]:
median_ranks_metadata_subset_ksizes = median_ranks_metadata_subset_scales.query('ksize in @ksizes')
median_ranks_metadata_subset_ksizes = median_ranks_metadata_subset_ksizes.dropna()
print(median_ranks_metadata_subset_ksizes.shape)
median_ranks_metadata_subset_ksizes.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 4))

sns.boxplot('ksize', 'nn_median_rank', data=median_ranks_metadata_subset_ksizes, 
               hue='scaled', palette='viridis')
sns.despine()

### Plot ksize as hue

In [ ]:
g = sns.FacetGrid(median_ranks_metadata_subset_ksizes, hue='ksize', col='trim', 
                  height=6, palette='magma', aspect=2)
g = g.map(sns.pointplot, 'scaled', 'nn_median_rank', estimator=np.median)
g = g.add_legend()

In [ ]:
from scipy import stats

In [ ]:
g = sns.FacetGrid(median_ranks_metadata_subset_ksizes, hue='ksize', col='trim', 
                  height=6, palette='magma', aspect=2)
g = g.map(sns.pointplot, 'scaled', 'nn_median_rank', estimator=stats.gmean)
g = g.add_legend()

In [ ]:
g = sns.FacetGrid(median_ranks_metadata_subset_ksizes, hue='ksize', col='trim', 
                  height=6, palette='magma', aspect=2)
g = g.map(sns.regplot, 'scaled', 'nn_median_rank', lowess=True)
g = g.add_legend()

Turn scatter off

In [ ]:
g = sns.FacetGrid(median_ranks_metadata_subset_ksizes, hue='ksize', col='trim', 
                  height=6, palette='magma', aspect=2)
g = g.map(sns.regplot, 'scaled', 'nn_median_rank', lowess=True, scatter=False)
g = g.add_legend()
g.set(xlim=(0, 5000))

In [ ]:
g = sns.FacetGrid(median_ranks_metadata_subset_ksizes, hue='ksize', col='trim', 
                  height=6, palette='magma', aspect=2)
g = g.map(sns.regplot, 'scaled', 'nn_median_rank')
g = g.add_legend()